<a href="https://colab.research.google.com/github/alfekka/datasets_ABBYY/blob/main/ml_train_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
nltk.download('wordnet')
nltk.download('punkt')
stemmer = SnowballStemmer('russian')

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans, DBSCAN

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import fowlkes_mallows_score

import seaborn as sns

from yellowbrick.text import FreqDistVisualizer, TSNEVisualizer

from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
data = pd.read_csv('/content/drive/My Drive/Corpus_project/basic_cleaning_tweets.csv')
data.head(10)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id,date,tweet
0,0,1323414585366450176,2020-11-02 23:59:59+00:00,подружка скинула песни алены швец теперь лежу ...
1,1,1323414583663579141,2020-11-02 23:59:59+00:00,Лол ивент рашки окончен. Из сотки сундуков мне...
2,2,1323414582162034695,2020-11-02 23:59:58+00:00,Юи солнышко
3,3,1323414578055794691,2020-11-02 23:59:57+00:00,Дайте поспааать
4,4,1323414572162830339,2020-11-02 23:59:56+00:00,хочу ему написать..
5,5,1323414566420779010,2020-11-02 23:59:55+00:00,меня так сильно не торкнуло но она всё ещё хор...
6,6,1323414562788544512,2020-11-02 23:59:54+00:00,"тааак я залезла на фикбук и очень надеюсь, что..."
7,7,1323414560909463554,2020-11-02 23:59:53+00:00,Завтра +7? Звучит стремно
8,8,1323414555876360192,2020-11-02 23:59:52+00:00,"Мне кажется, наша цель как можно больше дрочит..."
9,9,1323414555817553920,2020-11-02 23:59:52+00:00,"Что делать когда тебе кажется,что ты делаешь в..."


In [5]:
data.tweet[0]

'подружка скинула песни алены швец теперь лежу слушаю всё'

In [6]:
len(data.index)

702804

In [ ]:
data2 = []
for i in range(0, len(data.index)):

    # Punctuation removal
    table = str.maketrans(dict.fromkeys(string.punctuation))     
    twit = str(data.tweet[i])              
    print(type(twit))
    sentences = (twit.translate(table))

    # " '\n " removal
    words = sentences[:-3] 

    # Tokenization
    words = nltk.word_tokenize(words)

    # shrt words removal & lemmatization & stemming
    words_ = []
    for word in words:
        if len(word) > 2:   
            if not word.isnumeric():                                                  
                word1 = stemmer.stem(WordNetLemmatizer().lemmatize(word, pos='v'))          
                words_.append(word1)
    data2.append(words_)

# df = pd.DataFrame({'comment':data2, 'toxic':data['toxic']})   

In [ ]:
comparison = pd.DataFrame({'comments': data['tweet'], 'preprocessed comments': data2})
comparison.head()

,comments,preprocessed comments
0,подружка скинула песни алены швец теперь лежу ...,"[подружк, скинул, песн, ал, швец, тепер, леж, ..."
1,Лол ивент рашки окончен. Из сотки сундуков мне...,"[лол, ивент, рашк, оконч, сотк, сундук, мне, в..."
2,Юи солнышко,[солныш]
3,Дайте поспааать,"[дайт, поспа]"
4,хочу ему написать..,"[хоч, ем, напис]"


In [ ]:
comparison.to_csv('/content/drive/My Drive/Corpus_project/basic_cleaning_tweets_lem.csv', sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None)

In [6]:
comparison = pd.read_csv('/content/drive/My Drive/Corpus_project/basic_cleaning_tweets_lem.csv')


In [7]:
cvect = CountVectorizer(ngram_range=(1, 1), lowercase='true')   
tfidf_transformer = TfidfTransformer(norm= 'l2', use_idf= True)

In [9]:
data3 = comparison['comments'].astype(str).values.tolist()

data1_ = cvect.fit_transform(data3)
data_ = tfidf_transformer.fit_transform(data1_)

In [ ]:
kmeans = KMeans(n_clusters=2, init='k-means++', random_state=0).fit(data_)

In [ ]:
kmeans_pred = []
for label in kmeans.predict(data_):
    if label == 0:
      
        kmeans_pred.append(1)
    else:
        kmeans_pred.append(0)


In [35]:
with open('/content/drive/My Drive/Corpus_project/preds.txt' , 'w', encoding='utf-8') as dst:
  print(kmeans_pred, file=dst)

In [ ]:
preds_table = pd.DataFrame({'comments': data['tweet'], 'preprocessed comments': data2, 'preds':kmeans_pred})

In [37]:
preds_table.to_csv('/content/drive/My Drive/Corpus_project/preds_table.csv', sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None)

In [39]:
df = preds_table.sample(n=100)
df 

,comments,preprocessed comments,preds
149206,Мои сторис смотрит мой препод из универа. Инте...,"[мо, сторис, смотр, мо, препод, универ, интере...",1
501205,решительно настроена встречать новый год в пиж...,"[решительн, настро, встреча, нов, год, пижам, ...",1
664966,Честь — это мужественная стыдливость.,"[чест, эт, мужествен, стыдлив]",1
524176,"Тред для тредов, чтоб ничего не потерять","[тред, для, тред, чтоб, нич, потер]",0
328829,я так счастлива что смогла стать возможно для ...,"[так, счастлив, что, смогл, стат, возможн, для...",0
...,...,...,...
25540,Нашла совпадение в наличии вытамина Д в яичных...,"[нашл, совпаден, налич, вытамин, яичн, желтк, ...",0
3521,Почему так мало часов в суткааааааах??????????...,"[поч, так, мал, час, суткаааа]",1
257397,всех с новым годом,"[всех, нов]",1
459572,Это так ужасно Когда ты пишешь сообщение челов...,"[эт, так, ужасн, когд, пишеш, сообщен, человек...",0


In [ ]:
# df1['toxic'] = df['toxic']
df.to_csv('/content/drive/My Drive/Corpus_project/preds_table_random.csv', sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None)

In [5]:
df = pd.read_csv('/content/drive/My Drive/Corpus_project/preds_table_rando.csv', sep=',')
df

,Unnamed: 0,comments,preprocessed comments,preds,toxic
0,149206,Мои сторис смотрит мой препод из универа. Инте...,"['мо', 'сторис', 'смотр', 'мо', 'препод', 'уни...",1,1
1,501205,решительно настроена встречать новый год в пиж...,"['решительн', 'настро', 'встреча', 'нов', 'год...",1,0
2,664966,Честь — это мужественная стыдливость.,"['чест', 'эт', 'мужествен', 'стыдлив']",1,0
3,524176,"Тред для тредов, чтоб ничего не потерять","['тред', 'для', 'тред', 'чтоб', 'нич', 'потер']",0,0
4,328829,я так счастлива что смогла стать возможно для ...,"['так', 'счастлив', 'что', 'смогл', 'стат', 'в...",0,0
...,...,...,...,...,...
95,25540,Нашла совпадение в наличии вытамина Д в яичных...,"['нашл', 'совпаден', 'налич', 'вытамин', 'яичн...",0,0
96,3521,Почему так мало часов в суткааааааах??????????...,"['поч', 'так', 'мал', 'час', 'суткаааа']",1,0
97,257397,всех с новым годом,"['всех', 'нов']",1,0
98,459572,Это так ужасно Когда ты пишешь сообщение челов...,"['эт', 'так', 'ужасн', 'когд', 'пишеш', 'сообщ...",0,0


In [6]:
fowlkes_mallows_score(df['preds'], df['toxic']) 

0.5514251321475692

In [7]:
def accuracy_score_(labels__, labels___):
    score = 0
    for idx, label in enumerate(labels__):
        if label == labels___[idx]:
            score = score + 1

    return score/len(labels___)

In [8]:
accuracy_score_(df['preds'], df['toxic'])

0.59

In [9]:
print(classification_report(df['toxic'], df['preds']))

              precision    recall  f1-score   support

           0       0.80      0.41      0.54        59
           1       0.50      0.85      0.63        41

    accuracy                           0.59       100
   macro avg       0.65      0.63      0.58       100
weighted avg       0.68      0.59      0.58       100



In [10]:
confusion_matrix(df['toxic'], df['preds'])

array([[24, 35],
       [ 6, 35]])

из странного: токсичным классифицируется новый год и якутский